In [43]:
import numpy as np
import pandas as pd
import networkx as nx
from IPython.display import display, Image
from pathlib import Path
import matplotlib.pyplot as plt

import datetime

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [14]:
df_test = pd.read_csv('csv/test.csv')
info = pd.read_csv('csv/info.csv')
df_network = pd.read_csv('csv/network.csv')
news = pd.read_csv('csv/covid-19_news.csv')
sample_submit = pd.read_csv('csv/sample_submit.csv')
df_train = pd.read_csv('csv/train.csv')

In [15]:
df_test.date=df_test.date.astype('str')
df_test['Arrival'] = df_test['date'].str.cat(df_test['planArrival'],sep=' ')
df_test['Arrival'] = pd.to_datetime(df_test.Arrival)

df_train.date=df_train.date.astype('str')
df_train['Arrival'] = df_train['date'].str.cat(df_train['planArrival'],sep=' ')
df_train['Arrival'] = pd.to_datetime(df_train.Arrival)

In [16]:
df_train

,id,date,lineName,directionCode,trainNo,stopStation,planArrival,delayTime,Arrival
0,0,20191204,A,1,AFGKvJ,Rlfq,06:11,0.0,2019-12-04 06:11:00
1,1,20191204,A,1,AFGKvJ,coZB,06:13,0.0,2019-12-04 06:13:00
2,2,20191204,A,1,AFGKvJ,LMww,06:16,0.0,2019-12-04 06:16:00
3,3,20191204,A,1,AFGKvJ,VNyR,06:18,0.0,2019-12-04 06:18:00
4,4,20191204,A,1,AFGKvJ,jhlV,06:20,0.0,2019-12-04 06:20:00
...,...,...,...,...,...,...,...,...,...
1488880,1488880,20201022,D,2,zbKwea,BCRD,18:47,0.0,2020-10-22 18:47:00
1488881,1488881,20201022,D,2,zbKwea,AVjc,18:51,1.0,2020-10-22 18:51:00
1488882,1488882,20201022,D,2,zbKwea,uYlv,18:53,0.0,2020-10-22 18:53:00
1488883,1488883,20201022,D,2,zbKwea,wwYD,18:59,0.0,2020-10-22 18:59:00


In [22]:
df = pd.concat([df_train,df_test])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2748114 entries, 0 to 1259228
Data columns (total 10 columns):
 #   Column         Dtype         
---  ------         -----         
 0   id             int64         
 1   date           object        
 2   lineName       object        
 3   directionCode  int64         
 4   trainNo        object        
 5   stopStation    object        
 6   planArrival    object        
 7   delayTime      float64       
 8   Arrival        datetime64[ns]
 9   target         float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 230.6+ MB


# label encoding

In [30]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['lineName_cat'] = le.fit_transform(df['lineName'])
df['stopStation_cat'] = le.fit_transform(df['stopStation'])
df['Arrival_cat'] = le.fit_transform(df['Arrival'])

In [55]:

df_merge_train = df[df.target != 1]
df_merge_target = df_merge_train.delayTime
df_merge_train.drop(['target','id','date','planArrival','trainNo',
                     'delayTime','lineName','stopStation','Arrival'],
                    axis=1,inplace=True)
df_merge_test = df[df.target == 1]
sub_index = df_merge_test.id
df_merge_test.drop(['target','id','date','planArrival','trainNo',
                    'delayTime','lineName','stopStation','Arrival'],
                        axis=1,inplace=True)

C:\Users\kmtk49\Anaconda3\envs\TF2-GPU\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\kmtk49\Anaconda3\envs\TF2-GPU\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [32]:
df_merge_train

,directionCode,lineName_cat,stopStation_cat,Arrival_cat
0,1,0,37,907
1,1,0,53,909
2,1,0,22,912
3,1,0,44,914
4,1,0,72,916
...,...,...,...,...
1259223,2,3,42,48043
1259224,2,3,3,48046
1259225,2,3,2,48050
1259226,2,3,95,48052


# XGBoost
https://qiita.com/msrks/items/e3e958c04a5167575c41

In [40]:
train_x,test_x,train_y,test_y = train_test_split(df_merge_train,df_merge_target,
                                                test_size=0.2,shuffle=True)

In [42]:
reg = XGBRegressor()
reg.fit(train_x,train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [47]:
pred_train = reg.predict(train_x)
pred_test = reg.predict(test_x)

train_y.fillna(0,inplace=True)
test_y.fillna(0,inplace=True)
print(mean_squared_error(train_y, pred_train))
print(mean_squared_error(test_y, pred_test))

14.994338104578805
14.73918791240609


In [75]:
pred = reg.predict(df_merge_test)
pred = pd.DataFrame(pred)

In [82]:
sub_id = sub_index.reset_index().drop('index',axis=1)
sub = pd.concat([sub_id,pred],axis=1)

In [86]:
sub.to_csv("./sub/xgb01_20210104.csv", index=False, header=False)